Импорт необходимых библиотек

In [ ]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

Загрузка и подготовка данных

In [ ]:
# Загрузка данных Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Список классов (объединены некоторые классы для упрощения)
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Shirt/Coat', 'Bag', 'Ankle boot/Sandal/Sneaker']

# Нормализация изображений (приведение значений пикселей к диапазону [0, 1])
x_train = x_train / 255.0
x_test = x_test / 255.0

# Добавление 3 каналов (RGB) и изменение размера изображений до 32x32
x_train = np.stack([x_train] * 3, axis=-1)  # (60000, 28, 28, 3)
x_test = np.stack([x_test] * 3, axis=-1)  # (10000, 28, 28, 3)

x_train = tf.image.resize(x_train, (32, 32))  # Resize до 32x32
x_test = tf.image.resize(x_test, (32, 32))  # Resize до 32x32

Объединение классов

In [ ]:
# Объединение классов (переопределение меток)
y_train_copy = y_train.copy()
y_test_copy = y_test.copy()

for i in range(len(y_train)):
    if y_train_copy[i] == 5:
        y_train_copy[i] = 6
    elif y_train_copy[i] == 6:
        y_train_copy[i] = 4
    elif y_train_copy[i] == 7:
        y_train_copy[i] = 6
    elif y_train_copy[i] == 8:
        y_train_copy[i] = 5
    elif y_train_copy[i] == 9:
        y_train_copy[i] = 6

for i in range(len(y_test)):
    if y_test_copy[i] == 5:
        y_test_copy[i] = 6
    elif y_test_copy[i] == 6:
        y_test_copy[i] = 4
    elif y_test_copy[i] == 7:
        y_test_copy[i] = 6
    elif y_test_copy[i] == 8:
        y_test_copy[i] = 5
    elif y_test_copy[i] == 9:
        y_test_copy[i] = 6

y_train = np.array(y_train_copy)
y_test = np.array(y_test_copy)

Преобразование меток в one-hot encoding

In [ ]:
# Преобразование меток в one-hot encoding
y_train = keras.utils.to_categorical(y_train, 7)
y_test = keras.utils.to_categorical(y_test, 7)

Визуализация данных

In [ ]:
# Визуализация примеров изображений
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_train[i])
    plt.xlabel(class_names[np.argmax(y_train[i])])
plt.show()

Создание модели

In [ ]:
# Загрузка предобученной модели ResNet50 без верхнего слоя
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False  # Замораживаем веса

# Создание новой модели на основе ResNet50
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dense(96, activation='relu'),
    Dense(7, activation='softmax')
])

Компиляция модели

In [ ]:
# Компиляция модели
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Вывод структуры модели
model.summary()

Добавляем условия остановки обучения

In [ ]:
# Создание callback EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Метрика, которую будем отслеживать (потери на валидации)
    patience=30,          # Количество эпох без улучшения, после которых обучение остановится
    restore_best_weights=True  # Восстановление весов модели с лучшей эпохи
)

Обучение модели

In [ ]:
# Обучение модели (если модель еще не обучена)
if "fashion_trained_model_3.keras" not in os.listdir():
    history = model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test), callbacks=[early_stopping])
    model.save("fashion_trained_model_3.keras")

    # Построение графика точности на обучающей и тестовой выборках
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Test Accuracy')
    plt.title('Training and Test Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
else:
    # Если модель уже обучена, загружаем её
    model = keras.models.load_model('fashion_trained_model_3.keras')

Оценка модели

In [ ]:
# Оценка точности на тестовой выборке
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Визуализация результатов

In [ ]:
# Предсказание классов для тестовой выборки
predictions = model.predict(x_test)

# Визуализация примеров изображений с предсказанными и истинными классами
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[i])
    # Предсказание классов
    predicted_class = np.argmax(predictions[i])
    true_class = np.argmax(y_test[i])
    plt.xlabel(f'Predicted: {class_names[predicted_class]}, Actual: {class_names[true_class]}')
plt.show()